In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

: 

In [ ]:
conda install -p c:\Users\PKNU\anaconda3 ipykernel --update-deps --force-reinstall

: 

In [ ]:
df_hospital = pd.DataFrame(data)

# "진료과목내용명" 열에 "내과"와 "외과"가 모두 포함된 행 필터링
filtered_df = df_hospital[df_hospital['진료과목내용명'].str.contains('내과') & df_hospital['진료과목내용명'].str.contains('외과')]

# 필터링된 데이터프레임을 CSV 파일로 저장
filtered_df.to_csv('filtered_hospitals.csv', index=False)

print("CSV 파일이 성공적으로 저장되었습니다.")

: 

In [ ]:
import numpy as np
print(np.__version__)


: 

In [ ]:
import chardet
filename = r"거창_병원.csv" 
filename2 = r거창_경로당.csv"
with open(filename, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(1000))
with open(filename2, 'rb') as rawdata:
    result2 = chardet.detect(rawdata.read(1000))
print(result['encoding'], result2['encoding'])

: 

In [ ]:
df_hospital = pd.read_csv('거창_병원.csv', encoding='EUC-KR')
df_elderly = pd.read_csv('거창_경로당.csv', encoding='utf-8')

: 

In [ ]:
df_hospital

: 

In [ ]:
df_elderly

: 

In [ ]:
#하버사인 공식 이용

import math

def calculate_distance(elderly_lat, elderly_lon, hospital_lat, hospital_lon):
    R = 6371  
    dlat = math.radians(hospital_lat - elderly_lat)
    dlon = math.radians(hospital_lon - elderly_lon)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(elderly_lat)) * math.cos(math.radians(hospital_lat)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

distances = []

print(f"df_elderly 크기: {df_elderly.shape}")
print(f"df_hospital 크기: {df_hospital.shape}")

for i in range(len(df_elderly)):
    elderly_name = df_elderly.loc[i, '경로당']
    elderly_lat = df_elderly.loc[i, '위도']
    elderly_lon = df_elderly.loc[i, '경도']

    for j in range(len(df_hospital)):
        hospital_name = df_hospital.loc[j, '사업장명']
        hospital_lat = df_hospital.loc[j, '좌표정보(Y)']
        hospital_lon = df_hospital.loc[j, '좌표정보(X)']

        distance = calculate_distance(elderly_lat, elderly_lon, hospital_lat, hospital_lon)
        distances.append({
            '경로당': elderly_name,
            '병원': hospital_name,
            '거리': distance
        })

df_distances = pd.DataFrame(distances)

for elderly_name in df_distances['경로당'].unique():
    print(f"{elderly_name}과 병원 간의 거리")
    print(df_distances[df_distances['경로당'] == elderly_name][['병원', '거리']])
    print("\n")

output_file_path = '경로당별_병원_거리.csv'
df_distances.to_csv(output_file_path, index=False, encoding='euc-kr')

: 

In [ ]:
# 각 경로당별 가까운 5개의 병원 선택

df_top5 = df_distances.groupby('경로당').apply(lambda x: x.nsmallest(3, '거리')).reset_index(drop=True)

for elderly_name in df_top5['경로당'].unique():
    print(f"{elderly_name}과 병원 간의 가까운 3개 거리")
    print(df_top5[df_top5['경로당'] == elderly_name][['병원', '거리']])
    print("\n")

output_file_path_top5 = '경로당별_병원3개.csv'
df_top5.to_csv(output_file_path_top5, index=False, encoding='euc-kr')

: 

In [ ]:
# 각 경로당별 가까운 5개의 병원 거리 평균 계산

df_mean_distance = df_top5.groupby('경로당').agg(평균거리=('거리', 'mean')).reset_index()

print("경로당별 병원 평균 거리")
print(df_mean_distance)

output_file_path_mean = '경로당별_병원평균(3개)거리.csv'
df_mean_distance.to_csv(output_file_path_mean, index=False, encoding='euc-kr')

: 

In [ ]:
from folium.plugins import MarkerCluster
import base64
import os
import folium

df_elderly = df_elderly.merge(df_mean_distance, on='경로당')
m = folium.Map(location=[35.1796, 128.0816], zoom_start=12) 
marker_cluster = MarkerCluster().add_to(m)

for idx, row in df_elderly.iterrows():
    lat = row['위도']
    lon = row['경도']
    popup_text = f"<b>{row['경로당']}</b><br>평균거리: {row['평균거리']:.2f} km"

    bar_html = """
    <div style="width: 100%; height: 20px; background-color: lightgrey;">
        <div style="width: {0}%; height: 100%; background-color: blue;"></div>
    </div>
    """.format(row['평균거리'] / df_mean_distance['평균거리'].max() * 100)
    
    popup_content = f"{popup_text}<br>{bar_html}"
    
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

m.save('경로당별_평균(3개)거리_지도.html')

: 